In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "IMR"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Non Quasi-Circular Corrections

In the SEOBNR formalism, the evolution of the binary is broken into two independent phases, the inspiral-plunge phase and the merger-ringdown phase. During the inspiral-plunge phase, the waveform is evaluated from the EOB trajectory using factorized. resummed Post-Newtonian equations for circular orbits. However, the EOB trajectory displays deviations from circular behaviour closer to the transition between the two phases. In order to account for the non-circular behaviour and to ensure that the inspiral-plunge and merger-ringdown waveforms are smoothly connected, it is important to introduce Non Quasi-Circular (NQC) corrections to the PN waveform.

This is accomplished by introducting a correction to the factorized waveform of the form:

$$
h^{\rm insp-plunge}_{lm} = 
$$

# Step : Compute the q_i, p_i 

$$
q = \left\{ \frac{p_r^2}{r^2\Omega^2} , \frac{p_r^2}{r^2\Omega^2}\frac{1}{r} , \frac{p_r^2}{r^2\Omega^2}\frac{1}{r^{3/2}} \right\} \\
p = -\left\{ \frac{p_r}{r\Omega} , \frac{p_r^2}{r\Omega} \right\}
$$

In [2]:
%%writefile $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
time = dynamics_fine[:,0]
r = dynamics_fine[:,1]
omega_orb = dynamics_fine[:,6]
pr = dynamics_fine[:,3]
rOmega = r * omega_orb
q1 = pr*pr / (rOmega*rOmega)
q2 = q1 / r
q3 = q2 / np.sqrt(r)
p1 = -pr / rOmega
p2 = -p1 * pr * pr

Overwriting IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Compute amplitude and phase terms

In [3]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
amplitude = np.abs(h22_fine[:,1])
phase = np.unwrap(np.angle(h22_fine[:,1]))

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step Compute the interpolation intervals

In [4]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
idx = np.argmin(np.abs(time - t_attach))
N = 5
left = np.max((0, idx - N))
right = np.min((idx + N, len(time)))

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Interpolate Q_i and compute the Q matrix

In [5]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
Q1 = q1 * amplitude
Q2 = q2 * amplitude
Q3 = q3 * amplitude
intrp_Q1 = InterpolatedUnivariateSpline(time[left:right], Q1[left:right])
intrp_Q2 = InterpolatedUnivariateSpline(time[left:right], Q2[left:right])
intrp_Q3 = InterpolatedUnivariateSpline(time[left:right], Q3[left:right])
Q = np.zeros([3,3])
Q[:, 0] = intrp_q1LM.derivatives(t_attach)[:-1]
Q[:, 1] = intrp_q2LM.derivatives(t_attach)[:-1]
Q[:, 2] = intrp_q3LM.derivatives(t_attach)[:-1]

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Interpolate P_i and compute the Q matrix

In [6]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
intrp_P1 = InterpolatedUnivariateSpline(time[left:right], p1[left:right])
intrp_P2 = InterpolatedUnivariateSpline(time[left:right], p2[left:right])
P = np.zeros([2,2])
P[:, 0] = -intrp_p1.derivatives(t_attach)[1:-1]
P[:, 1] = -intrp_p2.derivatives(t_attach)[1:-1]

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Interpolate A and omega terms

pySEOBNR performs checks to ensure that there is no negative frequency

In [7]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
intrp_amp = InterpolatedUnivariateSpline(time[left:right], amplitude[left:right])
amps_inspiral = intrp_amp.derivatives(t_attach)[:-1]
intrp_phase = InterpolatedUnivariateSpline(time[left:right], phase[left:right])
omega, omegaDot = intrp_phase.derivatives(t_attach)[1:-1]
if omega * omegaDot > 0.0:
    omega = np.abs(omega)
    omegaDot = np.abs(omegaDot)
else:
    omega = np.abs(omega)
    omegaDot = -np.abs(omegaDot)
omegas_inspiral = [omega,omegaDot]

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Compute corresponding BOB terms

In [8]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
amps_BOB, omegas_BOB = v5HM_BOB_unoptimized_nqc_terms(t_attach,t_peak_strain,h22NRpeak,omega22NRpeak,omegaQNM,tau)

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Compute the NQC right hand sides


In [9]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
rhs_ai = np.array(amps_BOB) - np.array(amps_inspiral)
rhs_bi = np.array(omegas_BOB) - np.array(omegas_inspiral)

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Solve for a_i and b_i


In [10]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_coefficients.txt
res_ai = np.linalg.solve(Q,amps)
res_bi = np.linalg.solve(P,omegas)

Appending to IMR/v5HM-BOB_NQC_coefficients.txt


# Step : Write routine into python function

In [11]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_compute_nqc_coefficients.py"),"w") as output:
    output.write("import numpy as np\n")
    output.write( "from scipy.interpolate import InterpolatedUnivariateSpline\n")
    output.write( "from IMR.v5HM_BOB_unoptimized_nqc_terms import v5HM_BOB_unoptimized_nqc_terms\n")
    output.write( "def v5HM_BOB_unoptimized_compute_nqc_coefficients(t_peak_strain, t_attach, h22_fine, dynamics_fine, h22NRpeak, omega22NRpeak, omegaQNM, tau):\n")
    for line in list(open(os.path.join(Ccodesdir,"v5HM-BOB_NQC_coefficients.txt"),"r")):
        output.write("    %s\n" % line)
    output.write('    return {"a": res_ai, "b" : res_bi}')

# Step : Apply NQC corrections

# Step : Retrieve NQC coefficients

In [12]:
%%writefile $Ccodesdir/v5HM-BOB_NQC_corrections.txt
ai = nqc_coefficients['a']
bi = nqc_coefficients['b']

Overwriting IMR/v5HM-BOB_NQC_corrections.txt


# Step : compute q_i

In [13]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
r = dynamics[:,1]
omega_orb = dynamics[:,6]
pr = dynamics[:,3]
rOmega = r * omega_orb
q1 = pr*pr / (rOmega*rOmega)
q2 = q1 / r
q3 = q2 / np.sqrt(r)

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : compute amplitude correction

In [14]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
N22amp = 1 + q1*ai[0] + q2*ai[1] + q3*ai[2]

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : compute p_i

In [15]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
p1 = -pr / rOmega
p2 = -p1 * pr * pr

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : compute phase correction

In [16]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
N22phase = np.exp( 1j*( p1*bi[0] + p2*bi[1] ) )

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : compute NQC correction

In [17]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
N22 = N22amp*N22phase

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : apply the NQC correction

In [18]:
%%writefile -a $Ccodesdir/v5HM-BOB_NQC_corrections.txt
h22_inspiral_plunge = h22_full*N22

Appending to IMR/v5HM-BOB_NQC_corrections.txt


# Step : Write routine into python function

In [19]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_apply_nqc_corrections.py"),"w") as output:
    output.write("import numpy as np\n")
    output.write("def v5HM_BOB_apply_nqc_correction(nqc_corrections, h22_full, dynamics):\n")
    for line in list(open(os.path.join(Ccodesdir,"v5HM-BOB_NQC_corrections.txt"),"r")):
        output.write("    %s\n" % line)
    output.write('    return h22_inspiral_plunge')